In [ ]:
!pip install lightgbm
!pip install optuna
!pip install pandas
!pip install neptune-client neptune-contrib["monitoring"]
!pip install sklearn
!pip install numpy

In [1]:
import optuna
#import optuna.integration.lightgbm as lgb
import lightgbm as lgb
import itertools
import json
from sklearn.metrics import mean_squared_error
import neptune
import pandas as pd

import warnings

import numpy as np
warnings.filterwarnings("ignore")

In [2]:
def read_data(dataset_path, mode):
    print('Read dataset for ' + mode + ' from file ' + dataset_path + mode + '.csv')
    dataset = pd.read_csv(dataset_path + mode + '.csv')
    dataset = dataset.drop(['Unnamed: 0'], axis=1)
    return dataset.drop(['y'], axis=1), dataset['y']

In [3]:
import neptunecontrib.monitoring.optuna as opt_utils
neptune_monitor_optuna = opt_utils.NeptuneMonitor()

In [4]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [5]:
print('Loading data...')
x_train, y_train = read_data('', 'train')
x_eval, y_eval = read_data('', 'eval')

neptune.init('kowson/OLN')

used_params = []

Loading data...
Read dataset for train from file train.csv
Read dataset for eval from file eval.csv


In [6]:
print('Preparing LightGBM datasets...')
lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval = lgb.Dataset(x_eval, y_eval, reference=lgb_train)


Preparing LightGBM datasets...


In [ ]:
TAGS = ['lightgbm', 'data_v5', '200k_uniform', 'mape', 'relative', 'optimization', 'optuna']

def neptune_monitor():
    def callback(env):
        for name, loss_name, loss_value, _ in env.evaluation_result_list:
            neptune.send_metric('{}_{}'.format(name, loss_name), x=env.iteration, y=loss_value)
    return callback


In [ ]:
print("Training...")
try:
    with open('lgb_params.json') as data_file:    
        used_params = json.load(data_file)
except (FileNotFoundError, json.JSONDecodeError): 
    used_params = []
keys, values = zip(*params_dict.items())
for v in itertools.product(*values):
    experiment_params = dict(zip(keys, v))
    if any(param_dict == experiment_params for param_dict in used_params):
        continue  # skip already computed
    neptune.create_experiment(
        name='LightGBM regressor with relative doses', 
        params=experiment_params,
        tags=TAGS
    )
    gbm = lgb.train(
        params=experiment_params, 
        train_set=lgb_train, 
        valid_sets=[lgb_train, lgb_eval],
        verbose_eval=False,
        early_stopping_rounds=5,
        callbacks=[neptune_monitor()],
    )
    # PREDICT AND EVAL
    y_pred = gbm.predict(x_eval, num_iteration=gbm.best_iteration)
    error = mean_squared_error(y_eval, y_pred) ** 0.5
    print("RMSE of prediction is: {}".format(error))
    neptune.log_text('rmse', str(error))
    error = mean_absolute_percentage_error(y_eval, y_pred)
    print("MAPE of prediction is: {}".format(error))
    neptune.log_text('mape', str(error))
    neptune.stop()
    used_params.append(experiment_params)
    with open('lgb_params.json', 'w') as outfile:
        json.dump(used_params, outfile, sort_keys=True, indent=4)

In [7]:
def objective(trial):
    param = {
        'objective': 'regression',
        'metric': 'l2',
        'boosting_type': trial.suggest_categorical('boosting_type', ['gbdt', 'goss', 'dart']),
        'num_leaves': trial.suggest_int('num_leaves', 20, 256),
        'num_rounds': trial.suggest_int('num_rounds', 100, 700),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 0.5),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 20, 100),
        'verbose': 0,
    }
    gbm = lgb.train(
        params=param, 
        train_set=lgb_train, 
        valid_sets=[lgb_train, lgb_eval],
        verbose_eval=False,
        early_stopping_rounds=5,
    )
    # PREDICT AND EVAL
    y_pred = gbm.predict(x_eval, num_iteration=gbm.best_iteration)
    #error = mean_squared_error(y_eval, y_pred) ** 0.5
    #print("RMSE of prediction is: {}".format(error))
    #neptune.log_text('rmse', str(error))
    error = mean_absolute_percentage_error(y_eval, y_pred)
    print("MAPE of prediction is: {}".format(error))
    neptune.log_metric('mape_error', error)
    #neptune.stop()
    return error

In [8]:
print("Optimizing...")
neptune.create_experiment(
    name='Optuna optimization with relative doses', 
    tags=['optimization', 'optuna', 'lightgbm', 'data_v5', 'relative', 'uniform_200k', 'l2']
)
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=250, callbacks=[neptune_monitor_optuna])

Optimizing...
https://ui.neptune.ai/kowson/OLN/e/OLN-3190
MAPE of prediction is: 6.672050006002464


[I 2020-05-22 15:12:00,719] Finished trial#0 with value: 6.672050006002464 with parameters: {'boosting_type': 'goss', 'num_leaves': 214, 'num_rounds': 230, 'learning_rate': 0.0011054536505026786, 'min_data_in_leaf': 41}. Best is trial#0 with value: 6.672050006002464.


MAPE of prediction is: 2.8135853023804027


[I 2020-05-22 15:12:39,664] Finished trial#1 with value: 2.8135853023804027 with parameters: {'boosting_type': 'gbdt', 'num_leaves': 223, 'num_rounds': 437, 'learning_rate': 0.01907882816340135, 'min_data_in_leaf': 94}. Best is trial#1 with value: 2.8135853023804027.


MAPE of prediction is: 60.96925338894776


...

[I 2020-05-22 19:03:23,227] Finished trial#247 with value: 2.093020547219298 with parameters: {'boosting_type': 'gbdt', 'num_leaves': 217, 'num_rounds': 653, 'learning_rate': 0.0775245273839355, 'min_data_in_leaf': 53}. Best is trial#54 with value: 2.075449686609079.


MAPE of prediction is: 2.0992866807221002


[I 2020-05-22 19:04:04,147] Finished trial#248 with value: 2.0992866807221002 with parameters: {'boosting_type': 'gbdt', 'num_leaves': 212, 'num_rounds': 666, 'learning_rate': 0.10818208606545114, 'min_data_in_leaf': 61}. Best is trial#54 with value: 2.075449686609079.


MAPE of prediction is: 2.0967218133334664


[I 2020-05-22 19:04:46,855] Finished trial#249 with value: 2.0967218133334664 with parameters: {'boosting_type': 'gbdt', 'num_leaves': 206, 'num_rounds': 680, 'learning_rate': 0.08928490824029008, 'min_data_in_leaf': 64}. Best is trial#54 with value: 2.075449686609079.


In [ ]:
neptune.stop()

In [ ]:
!pip install joblib
joblib.dump(study, 'optuna-study.pkl')